In [1]:
# Setup stuff goes here
import utils

utils.setupOpenAIAPI()

# Text to Audo (TTA) Demostration

Converts text to a mp3 file with OpenAI's TTA model

In [ ]:
audioFileOutput = "audio_ouput.mp3"
text = "Hello World! Hear me speak!"

utils.textToAudioFile(text, audioFileOutput) # Convert example text to audio file as test

# Audio to Text (ATT) Demostration

Converts Audio file to text(ATT) 

In [ ]:
audioFileInput = "Audio Input Test.m4a" 

transcription = utils.audioToText(audioFileInput)
print(transcription)

# First Echo Attempt
Successfully detects human voice, starts recording when human speaks, stops recording when human speaks and then converts human speech to text and then text to audio and plays back audio the human spoke in OpenAI's TTA voice.

Not real time or useful in practical as only when finish recording does it translate to text and text to audio and only when audio is finished writting to file does it play back. this means that it is so slow and this is the subject of the next improvement

In [2]:
import os
import sounddevice as sd
import scipy.io.wavfile as wav
import numpy as np
import webrtcvad
from pydub import AudioSegment
from pydub.playback import play
import collections
import time

# Define audio parameters
fs = 16000 
frame_duration_ms = 30 

def record_audio_vad(filename):
    vad = webrtcvad.Vad(1)
    num_padding_frames = 10 
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    triggered = False
    voiced_frames = []
    print("Recording... Speak now.")

    # silence_threshold_seconds = 1.5 
    silence_threshold_seconds = 0 
    last_voice_timestamp = time.time()

    with sd.InputStream(samplerate=fs, channels=1, dtype='int16') as stream:
        while True:
            audio_chunk, overflowed = stream.read(int(fs * frame_duration_ms / 1000))
            is_speech = vad.is_speech(audio_chunk.tobytes(), fs)

            if is_speech:
                last_voice_timestamp = time.time()

            if not triggered:
                ring_buffer.append((audio_chunk, is_speech))
                num_voiced = len([f for f, speech in ring_buffer if speech])
                if num_voiced > 0.9 * ring_buffer.maxlen:
                    triggered = True
                    print("Recording started.")
                    voiced_frames.extend([f for f, s in ring_buffer])
                    ring_buffer.clear()
            else:
                voiced_frames.append(audio_chunk)
                ring_buffer.append((audio_chunk, is_speech))
                num_unvoiced = len([f for f, speech in ring_buffer if not speech])
                
                if num_unvoiced > 0.9 * (ring_buffer.maxlen) and (time.time() - last_voice_timestamp > silence_threshold_seconds):
                    print("Recording stopped.")
                    break

    audio_data = np.concatenate(voiced_frames, axis=0)
    wav.write(filename, fs, audio_data)
    print("Recording complete.")

def transcribe_audio(filename):
    return utils.audioToText(filename) 

def synthesize_speech(text, output_filename):
    utils.textToAudioFile(text, output_filename) 

def play_audio(filename):
    audio = AudioSegment.from_mp3(filename)
    play(audio)

input_audio_file = "input6.wav"
output_audio_file = "output6.mp3"

record_audio_vad(input_audio_file)
transcription = transcribe_audio(input_audio_file)
print("Transcription:", transcription)

synthesize_speech(transcription, output_audio_file)
play_audio(output_audio_file)


Recording... Speak now.
Recording started.
Recording stopped.
Recording complete.
Transcription: Hello.


# Second Echo Attempt

In [1]:
from google.cloud import speech_v1 as speech
from google.oauth2 import service_account

def run_quickstart() -> speech.RecognizeResponse:
    """Runs the Speech-to-Text quickstart with credentials."""

    # **Move credentials definition outside the function**
    credentials = service_account.Credentials.from_service_account_file(
    'google_credential.json')
    # credentials = cr.from_service_account_file("google_credential.json")

    # Create a Speech client with credentials
    client = speech.SpeechClient(credentials=credentials)

    # The name of the audio file to transcribe
    gcs_uri = "gs://cloud-samples-data/speech/brooklyn_bridge.raw"  # Replace with your audio file URI (e.g., local path)

    audio = speech.RecognitionAudio(uri=gcs_uri)

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",  # Change to your desired language code
    )

    # Detects speech in the audio file
    response = client.recognize(config=config, audio=audio)

    for result in response.results:
        print(f"Transcript: {result.alternatives[0].transcript}")

if __name__ == "__main__":
    run_quickstart()

Transcript: how old is the Brooklyn Bridge


In [ ]:
import queue
import sys
import sounddevice as sd
from google.cloud import speech_v1 as speech
from google.oauth2 import service_account

# Audio recording parameters
RATE = 16000
# CHUNK = int(RATE / 10)  # 100ms
CHUNK = int(RATE)  # 100ms

# Initialize the queue to hold audio data
q = queue.Queue()

# Define the callback function to process audio data
def callback(indata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    q.put(bytes(indata))

def main():
    credentials = service_account.Credentials.from_service_account_file('google_credential.json')

    client = speech.SpeechClient(credentials=credentials)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code="en-US",
        enable_automatic_punctuation=True
    )
    # streaming_config = speech.StreamingRecognitionConfig(config=config, interim_results=True)
    streaming_config = speech.StreamingRecognitionConfig(config=config, interim_results=False)

    def generator():
        while True:
            data = q.get()
            if data is None:
                break
            yield speech.StreamingRecognizeRequest(audio_content=data)

    with sd.RawInputStream(samplerate=RATE, blocksize=CHUNK, dtype='int16',
                           channels=1, callback=callback):
        print("Listening (press Ctrl+C to stop)...")
        requests = generator()
        responses = client.streaming_recognize(config=streaming_config, requests=requests)

        try:
            for response in responses:
                for result in response.results:
                    if result.is_final:
                        print(f"Transcript: {result.alternatives[0].transcript}")
                    else:
                        print(f"Intermediate transcript: {result.alternatives[0].transcript}", end='\r')
        except KeyboardInterrupt:
            print("\nExiting...")
            q.put(None)

if __name__ == "__main__":
    main()

Listening (press Ctrl+C to stop)...
Transcript: Now it's dinner time.


In [ ]:
import io
from google.cloud import texttospeech
from google.oauth2 import service_account
from pydub import AudioSegment
import simpleaudio as sa

def text_to_speech(text):
    # Define the credentials file
    credentials = service_account.Credentials.from_service_account_file('google_credential.json')

    # Initialize the Text-to-Speech client
    client = texttospeech.TextToSpeechClient(credentials=credentials)

    # Set the text input to be synthesized
    synthesis_input = texttospeech.SynthesisInput(text=text)

    # Build the voice request
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )

    # Select the type of audio file you want returned
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    # Perform the text-to-speech request
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    # Use pydub to load the audio content
    audio = AudioSegment.from_file(io.BytesIO(response.audio_content), format="mp3")

    # Play the audio using simpleaudio
    play_obj = sa.play_buffer(
        audio.raw_data,
        num_channels=audio.channels,
        bytes_per_sample=audio.sample_width,
        sample_rate=audio.frame_rate
    )

    # Wait for playback to finish before exiting
    play_obj.wait_done()

if __name__ == "__main__":
    text = "Hello, world! This is a text-to-speech test using Google Cloud."
    text_to_speech(text)

In [ ]:
import io
from google.cloud import texttospeech
from google.oauth2 import service_account
from pydub import AudioSegment
import simpleaudio as sa

def text_to_speech(text):
    # Define the credentials file
    credentials = service_account.Credentials.from_service_account_file('google_credential.json')

    # Initialize the Text-to-Speech client
    client = texttospeech.TextToSpeechClient(credentials=credentials)

    # Set the text input to be synthesized
    synthesis_input = texttospeech.SynthesisInput(text=text)

    # Build the voice request
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )

    # Select the type of audio file you want returned
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    # Perform the text-to-speech request
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    # Use pydub to load the audio content
    audio = AudioSegment.from_file(io.BytesIO(response.audio_content), format="mp3")

    # Play the audio using simpleaudio
    play_obj = sa.play_buffer(
        audio.raw_data,
        num_channels=audio.channels,
        bytes_per_sample=audio.sample_width,
        sample_rate=audio.frame_rate
    )

    # Wait for playback to finish before exiting
    play_obj.wait_done()

if __name__ == "__main__":
    text = "Dinosaurs exist everywhere in the world!"
    text_to_speech(text)

In [2]:
import io
from google.cloud import texttospeech
from google.oauth2 import service_account
from pydub import AudioSegment
import pyaudio
import time

def play_audio(audio_segment):
    # Initialize PyAudio
    p = pyaudio.PyAudio()

    # Open a .Stream with the appropriate settings
    stream = p.open(format=p.get_format_from_width(audio_segment.sample_width),
                    channels=audio_segment.channels,
                    rate=audio_segment.frame_rate,
                    output=True)

    # Write the raw audio data to the stream
    stream.write(audio_segment.raw_data)

    # Close the stream and terminate PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()

def text_to_speech(text):
    # Define the credentials file
    credentials = service_account.Credentials.from_service_account_file('google_credential.json')

    # Initialize the Text-to-Speech client
    client = texttospeech.TextToSpeechClient(credentials=credentials)

    # Set the text input to be synthesized
    synthesis_input = texttospeech.SynthesisInput(text=text)

    # Build the voice request
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )

    # Select the type of audio file you want returned
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    # Perform the text-to-speech request
    response = client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    # Use pydub to load the audio content
    audio = AudioSegment.from_file(io.BytesIO(response.audio_content), format="mp3")

    # Play the audio
    play_audio(audio)

if __name__ == "__main__":
    text = "Now that the dinosaurs have left our house we can eat safely without being attacked!!!"
    startTime = time.time()
    text_to_speech(text)
    latency = time.time() - startTime
    print(f"latency:{latency} seconds")

latency:6.948406934738159 seconds


In [ ]:
import openai
import os

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")

def chatbot():
  # Create a list to store all the messages for context
  messages = [
    {"role": "system", "content": "You are a helpful assistant."},
  ]

  # Keep repeating the following
  while True:
    # Prompt user for input
    message = input("User: ")

    # Exit program if user inputs "quit"
    if message.lower() == "quit":
      break

    # Add each new message to the list
    messages.append({"role": "user", "content": message})

    # Request gpt-3.5-turbo for chat completion
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages
    )

    # Print the response and add it to the messages list
    chat_message = response['choices'][0]['message']['content']
    print(f"Bot: {chat_message}")
    messages.append({"role": "assistant", "content": chat_message})

if __name__ == "__main__":
  print("Start chatting with the bot (type 'quit' to stop)!")
  chatbot()